In [2]:

import pyomo.environ as pe
import math

# Data
factories = ["IJmuiden", "Segal", "South Wales"]
rebars = {"A": {"length": 2.4}, "B": {"length": 3.6}, "C": {"length": 4.2}}
longbars = {1: {"length": 9}, 2: {"length": 12}}
diameter = 0.057  # in meters
density = 7.85  # tons/m^3
production_capacity = {"IJmuiden": 12, "Segal": 10, "South Wales": 28}
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]
periods = [1, 2, 3, 4]

demand = {
    "A": {
        "Bochum": [2, 6, 5, 3],
        "Boenen": [4, 8, 5, 10],
        "Dortmund": [2, 7, 6, 5],
        "Gelsenkirchen": [5, 5, 5, 5],
        "Hagen": [19, 23, 25, 16],
        "Iserlohn": [13, 19, 17, 14],
        "Neuss": [20, 16, 14, 26],
        "Schwerte": [4, 5, 3, 4]
    },
    "B": {
        "Bochum": [4, 5, 7, 8],
        "Boenen": [5, 8, 12, 13],
        "Dortmund": [4, 5, 8, 10],
        "Gelsenkirchen": [9, 10, 6, 6],
        "Hagen": [15, 33, 31, 33],
        "Iserlohn": [22, 26, 20, 27],
        "Neuss": [12, 23, 30, 30],
        "Schwerte": [2, 8, 2, 6]
    },
    "C": {
        "Bochum": [6, 7, 7, 7],
        "Boenen": [6, 10, 15, 12],
        "Dortmund": [7, 6, 4, 12],
        "Gelsenkirchen": [10, 9, 9, 10],
        "Hagen": [12, 35, 33, 38],
        "Iserlohn": [14, 25, 23, 24],
        "Neuss": [22, 32, 31, 31],
        "Schwerte": [5, 6, 7, 2]
    }
}

fixed_cost = {"IJmuiden": 130, "Segal": 150, "South Wales": 100}
v = 0.5  # variable cost €/km/tonne
distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Precompute parameters
w_r = {r: rebars[r]["length"] for r in rebars}
w_l = {l: longbars[l]["length"] for l in longbars}
area = math.pi * (diameter / 2) ** 2


# possible number of long bars per long bar type per factory 
N = {
    1: {"IJmuiden": 67, "Segal": 56, "South Wales": 156},
    2: {"IJmuiden": 50, "Segal": 42, "South Wales": 117}
}

possible_longbars = {} 

for l in longbars:
    possible_longbars[l] = {} 
    for f in factories:
        possible_longbars[l][f] = list(range(1, N.get(l, {}).get(f, 0) + 1))
  

In [8]:
# Question 1
# Create the model
model = pe.ConcreteModel()

# Define sets
model.F = pe.Set(initialize=factories)
model.R = pe.Set(initialize=rebars.keys())
model.L = pe.Set(initialize=longbars.keys())
model.A = pe.Set(initialize=customers)
model.T = pe.Set(initialize=periods)


valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model.x = pe.Var(model.R, model.L_N_F, model.T, domain=pe.NonNegativeIntegers)
model.y = pe.Var(model.L_N_F, model.T, domain=pe.Binary)
model.z = pe.Var(model.F, model.A, model.T, domain=pe.Binary)

# Constraints using ConstraintList and for loops
# 1. Cutting constraints
model.cutting_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        for l in model.L:
            for n in possible_longbars[l][f]:
                expression = sum(model.x[r, l, n, f, t] * w_r[r] for r in model.R) <= model.y[l, n, f, t] * w_l[l]
                model.cutting_cnstr.add(expression)

# 2. Production capacity
model.capacity_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        expression = sum(model.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model.L) * density * area <= production_capacity[f]
        model.capacity_cnstr.add(expression)

# 3. Demand satisfaction
model.demand_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        for r in model.R:
            expression = sum(model.x[r, l, n, f, t] for n in possible_longbars[l][f] for l in model.L) >= sum(model.z[f, a, t] * demand[r][a][t-1] for a in model.A)
            model.demand_cnstr.add(expression)

# 4. Single sourcing
model.single_sourcing_cnstr = pe.ConstraintList()
for a in model.A:
    for t in model.T:
        expression = sum(model.z[f, a, t] for f in model.F) == 1
        model.single_sourcing_cnstr.add(expression)

# Objective function
objExpr = sum(model.z[f, a, t] * (fixed_cost[f] +  v * distance[a][f] * sum(demand[r][a][t-1] * w_r[r] for r in model.R) * density * area) for f in model.F for a in model.A for t in model.T)  
model.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result = solver.solve(model, tee=True, options={'TimeLimit':36})

# Display results
print(f"\nSolver Status: {result.solver.status}")
print(f"Termination Condition: {result.solver.termination_condition}")
if result.solver.status == pe.SolverStatus.ok and result.solver.termination_condition == pe.TerminationCondition.optimal:
    print(f"Objective value (Total Cost): {pe.value(model.obj):.2f} €")
    print("\nFactory assignments (z[f,a,t] = 1):")
    for f in model.F:
        for a in model.A:
            for t in model.T:
                if pe.value(model.z[f, a, t]) > 0.5:
                    print(f"z[{f}, {a}, {t}] = 1")
    print("\nRebars cut (x[r,l,n,f,t] > 0):")
    for r in model.R:
        for l in model.L:
            for f in model.F:
                for n in possible_longbars[l][f]:
                    for t in model.T:
                        if pe.value(model.x[r, l, n, f, t]) > 0:
                            print(f"x[{r}, {l}, {n}, {f}, {t}] = {pe.value(model.x[r, l, n, f, t]):.0f}")
    print("\nLong bars used (y[l,n,f,t] > 0):")
    for l in model.L:
        for f in model.F:
            for n in possible_longbars[l][f]:
                for t in model.T:
                    if pe.value(model.y[l, n, f, t]) > 0:
                        print(f"y[{l}, {n}, {f}, {t}] = {pe.value(model.y[l, n, f, t]):.0f}")
else:
    print("No optimal solution found.")

Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpknh9pupk.pyomo.lp
Reading time = 0.02 seconds
x1: 2032 rows, 7904 columns, 14880 nonzeros
Set parameter TimeLimit to value 36
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  36

Optimize a model with 2032 rows, 7904 columns and 14880 nonzeros
Model fingerprint: 0x1d4d1c58
Variable types: 0 continuous, 7904 integer (2048 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [2e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 718 rows and 2816 columns
Presolve time: 1.16s
Presolved: 1314 rows, 5088 columns, 10320 nonzeros
Variable types: 0 continuous, 5088 integer (1326 binary)

Root relaxation: objective 1.612448e+04, 1991 iterat

In [39]:
# New data for 2: Inventory capacities (from Table 7)
inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}

# Defining big M
# List to store total demand per customer
total_demand_per_customer = []

# Loop over each customer
for a in customers:
    total_demand = sum(demand[r][a][t-1] for r in rebars for t in periods)  # Sum over all rebars and periods
    total_demand_per_customer.append(total_demand)

# Get the maximum demand across all customers
M = max(total_demand_per_customer)


model2 = pe.ConcreteModel()

# Initialize sets for model 2
model2.F = pe.Set(initialize=factories)
model2.R = pe.Set(initialize=rebars.keys())
model2.L = pe.Set(initialize=longbars.keys())
model2.A = pe.Set(initialize=customers)
model2.T = pe.Set(initialize=periods)

valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model2.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model2.x = pe.Var(model2.R, model2.L_N_F,  model2.T, domain=pe.NonNegativeIntegers)
model2.y = pe.Var(model2.L_N_F, model2.T, domain=pe.Binary)
model2.z = pe.Var(model2.F, model2.A, model2.T, domain=pe.Binary)

# Add new variables to model2
model2.i = pe.Var(model2.A, model2.R, model2.T, domain=pe.NonNegativeIntegers)
model2.q = pe.Var(model2.R, model2.F, model2.A, model2.T, domain=pe.NonNegativeIntegers)


# Constraints 
# 1. Cutting constraints - same as Q1
model2.cutting_cnstr = pe.ConstraintList()
for f in model2.F:
    for t in model2.T:
        for l in model2.L:
            for n in possible_longbars[l][f]:
                expression = sum(model2.x[r, l, n, f, t] * w_r[r] for r in model2.R) <= model2.y[l, n, f, t] * w_l[l]
                model2.cutting_cnstr.add(expression)

# 2. Production capacity - same as Q1
model2.capacity_cnstr = pe.ConstraintList()
for f in model2.F:
    for t in model2.T:
        expression = sum(model2.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model2.L) * density * area <= production_capacity[f]
        model2.capacity_cnstr.add(expression)

# Factory output
model2.factory_output = pe.ConstraintList()
for f in model2.F:
    for r in model2.R:
        for t in model2.T:
            expression = sum(model2.q[(r, f, a, t)] for a in model2.A) == sum(model2.x[(r, l, n, f, t)] for n in possible_longbars[l][f] for l in model2.L)
            model2.factory_output.add(expression)

# Link z to q
model2.enforce_factory_usage = pe.ConstraintList()
for f in model2.F:
    for a in model2.A:
        for t in model2.T:
            expression = sum(model2.q[(r, f, a, t)] for r in model2.R) <= M * model2.z[f, a, t]
            model2.enforce_factory_usage.add(expression)

# Demand 
model2.demand = pe.ConstraintList()
for a in model2.A:
    for r in model2.R:
        for t in model2.T:
            shipped = sum(model2.q[(r, f, a, t)] for f in model2.F) 
            if t == 1:
                expression = shipped - model2.i[(a,r,t)] == demand[r][a][t-1] 
                model2.demand.add(expression)
            else:
                expression = shipped - model2.i[(a,r,t)] + model2.i[(a,r,t-1)] == demand[r][a][t-1] 
                model2.demand.add(expression)

# Single sourcing
model2.single_sourcing_cnstr = pe.ConstraintList()
for a in model2.A:
    for t in model2.T:
        expression = sum(model2.z[(f,a,t)] for f in model2.F) <= 1
        model2.single_sourcing_cnstr.add(expression)

# Inventory
model2.inventory = pe.ConstraintList()
for a in model2.A:
    for t in model2.T:
        expression = sum(model2.i[(a,r,t)]* w_r[r] for r in model2.R) * density * area <= inventory_capacity[a]
        model2.inventory.add(expression)


# Objective function
objExpr = sum(model2.z[f, a, t] * (fixed_cost[f] +  v * distance[a][f] * sum(model2.q[(r, f, a, t)] * w_r[r] for r in model2.R) * density * area) for f in model2.F for a in model2.A for t in model2.T)  
model2.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result2 = solver.solve(model2, tee=True, options={'TimeLimit':3600})

# Display results
print(f"\nSolver Status: {result2.solver.status}")
print(f"Termination Condition: {result2.solver.termination_condition}")
if result2.solver.status == pe.SolverStatus.ok and result2.solver.termination_condition == pe.TerminationCondition.optimal:
    print(f"Objective value (Total Cost): {pe.value(model2.obj):.2f} €")
    print("\nFactory assignments (z[f,a,t] = 1):")
    for f in model2.F:
        for a in model2.A:
            for t in model2.T:
                if pe.value(model2.z[f, a, t]) > 0.5:
                    print(f"z[{f}, {a}, {t}] = 1")
    print("\nQuantities shipped (q[r,f,a,t] > 0):")
    for r in model2.R:
        for f in model2.F:
            for a in model2.A:
                for t in model2.T:
                    if pe.value(model2.q[r,f,a,t]) > 0:
                        print(f"q[{r}, {f}, {a}, {t}] = {pe.value(model2.q[r,f,a,t]):.0f}")
    print("\nRebars cut (x[r,l,n,f,t] > 0):")
    for r in model2.R:
        for l in model2.L:
            for f in model2.F:
                for n in possible_longbars[l][f]:
                    for t in model2.T:
                        if pe.value(model2.x[r, l, n, f, t]) > 0:
                            print(f"x[{r}, {l}, {n}, {f}, {t}] = {pe.value(model2.x[r, l, n, f, t]):.0f}")
    print("\nLong bars used (y[l,n,f,t] > 0):")
    for l in model2.L:
        for f in model2.F:
            for n in possible_longbars[l][f]:
                for t in model2.T:
                    if pe.value(model2.y[l, n, f, t]) > 0:
                        print(f"y[{l}, {n}, {f}, {t}] = {pe.value(model2.y[l, n, f, t]):.0f}")
    
else:
    print("No optimal solution found.")


Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpvrivgd8t.pyomo.lp
Reading time = 0.02 seconds
x1: 2256 rows, 8288 columns, 15816 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 2256 rows, 8288 columns and 15816 nonzeros
Model fingerprint: 0x4f785dd6
Model has 288 quadratic objective terms
Variable types: 0 continuous, 8288 integer (2048 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+02]
  Objective range  [1e+02, 2e+02]
  QObjective range [7e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 280 rows and 1120 columns
Presolve time: 0.03s
Presolved: 2264 rows, 7456 columns, 15560 nonzeros
Variable types: 0 continuous, 74